# Demonstration of the PYME camera map infrastructure

## With applications to test CMOS image algorithms

In [1]:
# some useful imports
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# import our io and map handling routines
from PYME.localization.remFitBuf import CameraInfoManager
ci = CameraInfoManager()

import PYME.Analysis.gen_sCMOS_maps as gmaps
from PYME.IO.MetaDataHandler import NestedClassMDHandler

md = NestedClassMDHandler()

### Making a test metadata structure

Here we make a metadata structure from scratch to test the infrastructure of reading maps which matches in terms of camera data and ROI size etc.

For actual data this metadata should be directly supplied by the metadata stored with the acquired data.

In [7]:
def ROISZscale(md,refsize=7,refscale=0.1,minsize=5):
    roisize = max(int(refsize*refscale/md['voxelsize.x']+0.9),minsize)
    print 'ROI half size is %d' % roisize
    return roisize

md['voxelsize.x'] = 0.1
md['voxelsize.y'] = 0.1
md['voxelsize.z'] = 0.05

#what ROI size should we use
ROI_HALF_SIZE = ROISZscale(md)
ROI_SIZE = 2*ROI_HALF_SIZE + 1

md['Camera.ADOffset'] = 100.0
md['Camera.EMGain'] = 1.0
md['Camera.ElectronsPerCount'] = 0.5
md['Camera.NoiseFactor'] = 1.0 #ignored / calculated de-novo for simulation
md['Camera.ReadNoise'] = 1.4 #ignored for sCMOS
md['Camera.TrueEMGain'] = 1.0

#NB ROI width and height in metadata must match the simulation ROI size
md['Camera.ROIHeight'] = ROI_SIZE
md['Camera.ROIPosX'] = 415
md['Camera.ROIPosY'] = 166
md['Camera.ROIWidth'] = ROI_SIZE

md['Camera.SerialNumber'] = 'VSC-02858'

md['Camera.IntegrationTime'] = 0.1

ROI half size is 7


In [8]:
try:
    darkname = gmaps.mkDefaultPath('dark',md,create=False)
except ValueError:
    print 'calibration dir for camera does not exist'
try:
    varname = gmaps.mkDefaultPath('variance',md,create=False)
except ValueError:
    print 'calibration dir for camera does not exist'

print('Dark map path is %s' % darkname)
print('Variance map path is %s' % varname)
md['Camera.DarkMapID'] = darkname
md['Camera.VarianceMapID'] = varname

Dark map path is /Users/csoe002/PYMEData/CALIBRATION/VSC-02858/dark_100ms.tif
Variance map path is /Users/csoe002/PYMEData/CALIBRATION/VSC-02858/variance_100ms.tif


In [9]:
# based on all of this, the 'gain' according to the *** paper
# should be
#     gain = 1.0/(flat*mdh['Camera.ElectronsPerCount'])

# we need to figure out what we actually need, in terms of units
gain = 1.0/(ci.getFlatfieldMap(md)*md['Camera.ElectronsPerCount']) # in units of ADUperElectron
var = ci.getVarianceMap(md) # in electron^2
offset = ci.getDarkMap(md) # in ADUs


filename == /Users/csoe002/PYMEData/CALIBRATION/VSC-02858/variance_100ms.tif
/Users/csoe002/PYMEData/CALIBRATION/VSC-02858/variance_100ms.tif
[2048, 2048, 1, 1]
[2048, 2048, 1, 1]
filename == /Users/csoe002/PYMEData/CALIBRATION/VSC-02858/dark_100ms.tif
/Users/csoe002/PYMEData/CALIBRATION/VSC-02858/dark_100ms.tif
[2048, 2048, 1, 1]
[2048, 2048, 1, 1]


In [21]:
# this one we only need for interactive checks in a separate window - very convenient as we share the kernal and namespace for quick code evaluations
# %qtconsole